In [1]:
import numpy as np
import h5py
from numpy import linalg as linalg
import matplotlib.pyplot as plt
#read data
filename = "data-Mini Project 2.h5"
with h5py.File(filename, "r") as f:
    f.keys()
    trX = np.array(f["trX"])
    trY = np.array(f["trY"])
    tstX  = np.array(f["tstX"])
    tstY  = np.array(f["tstY"])

In [2]:
#normalize data
trX = (trX - np.mean(trX, axis=0)) / np.std(trX, axis=0)
tstX = (tstX - np.mean(tstX, axis=0)) / np.std(tstX, axis=0)

In [3]:
#mix the data since data is distributed uniformly
ahsen = np.random.permutation(600)
tstX_rand=tstX[ahsen]
tstY_rand=tstY[ahsen]
assen =np.random.permutation(3000)
trX_rand=trX[assen]
trY_rand = trY[assen]

In [4]:
#initialize paramteres
N=50
bs = 30
learning_rate = 0.003
weight_ih = np.random.uniform(low=-0.1,high=0.1,size=(N,3))
weight_hh = np.random.uniform(low=-0.1,high=0.1,size=(N,N))
weight_ho = np.random.uniform(low=-0.1,high=0.1,size=(6,N))
b = np.random.uniform(low=-0.1,high=0.1,size=(N,1)) #bias for wih
c = np.random.uniform(low=-0.1,high=0.1,size=(6,1)) #bias for who
weight_hh_up = np.zeros((N,N))
weight_ih_up = np.zeros((N,3))
weight_ho_up = np.zeros((6,N))
c_updated = np.zeros((6,1))
b_updated = np.zeros((N,1))

In [5]:
#activation functions and their derivatives
def sigmoid(v):
    return np.ones((len(v),1))/(np.ones((len(v),1))+ np.exp(-v)) 
def sigmoid_derivative(x):
    return x * (1 - x)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2
#forward propagation
def f_propagation(weight_ih,weight_hh,weight_ho,b,c,N,X):
    #initialize an empty array to hold previous values of outputs and h 
    hlist = np.zeros((len(X)+1,N,1))
    ylist = np.zeros((len(X),6,1))
    memo = np.zeros((N,1)) #previous data from forward (t-1)
    for t in range(150): #find for each 150 section
        vh= np.matmul(weight_ih,X[t].reshape((3,1)))
        vh = vh +np.matmul(weight_hh,memo)+ b 
        hidden = np.tanh(vh)
        vo = np.dot(weight_ho,hidden) + c
        out = sigmoid(vo)
        ylist[t] = out #add final outputs to array
        memo = hidden #holf temporary data
        hlist[t+1] = memo #add temp to later value of h
    return ylist

In [6]:
#training begins first forward then back 
for e in range(6): #epoch number
    print("Epoch", e+1) #track each epoch
    equal=0
    mean_error=0  #initialize mena error to 0 in every epoch 
    for element in range(150): #lenght of one data
        desiredifference= trY_rand[element].reshape((1,6)) #take training y data
        X=trX_rand[element] #takr training X data (line by line)
        hlist = np.zeros((150,N,1)) #assign an empty array to hold all h values
        ylist = np.zeros((150,6,1)) #assign an empty array to hold all output values
        memo = np.zeros((N,1)) #memory hold h(t-1)
        mean_error = 0
        for num in range(150//bs): #divide to batches
            #forward_propagation
            #same with def f_propagation
            for r in range(bs*num,bs*(num+1)):
                vh1=np.matmul(weight_ih, X[r].reshape((3,1)))
                vh = vh1 + np.matmul(weight_hh,hlist[r-1]) + b
                hidden = np.tanh(vh)
                vo = np.matmul(weight_ho,hidden) + c
                out = sigmoid(vo)
                ylist[r] = out
                hlist[r] = hidden #temp data
                #entropy formula
                total_error = -np.matmul(desiredifference,np.log(abs(out+1e-8)))-np.matmul((np.ones((1,6))-desiredifference),np.log(abs(np.ones((6,1))-out)+1e-8))
                #take mean of the total error 
                total_error=total_error.mean()
                #total_error = -np.matmul(desiredifference,np.log(out))-np.matmul(np.ones(shape=(1,6))-desiredifference,np.log(np.ones(shape=(6,1))-out))
                
                if not np.isnan(total_error): #to avoid possible nan values check if it is nan or not
                    mean_error = mean_error+ total_error/150 #mena_error divided by length of x
                #else:
                    #print("Value is nan")
                    
                #backpropagation begins (bptt)
                
                difference =ylist[bs*(num+1)-1] - desiredifference.reshape((6,1)) #difference=d-o ##(desired- output)
                diff = np.matmul(weight_ho.reshape((50,6)),difference)
                weight_ho_up = weight_ho_up + np.matmul(difference,hlist[bs*(num+1)-1].reshape((1,50))) #easiest to update since no (t-1) dependancy
                c_updated = c_updated+ difference #also update biases by using d-o (after who)
                
                #tanh derivative is needed to update other weigths and biases
                temp=(np.ones((1,N))-np.square(hlist[bs*(num+1)-1].reshape((1,50)))).flatten() #take the las value of hlist subtract by one 
                back_inp = np.matmul(np.diag(temp),diff) 
                b_updated =b_updated + back_inp #update bias for who before
                weight_hh_up =weight_hh_up+ np.matmul(back_inp,hlist[bs*(num+1)-2].reshape((1,50)))
                weight_ih_up = weight_ih_up+ np.matmul(back_inp,X[bs-1+num*bs].reshape((1,3)))
                for j in range(bs*(num+1)-2,bs*num,-1): #going back
                    difference = ylist[j] - desiredifference.reshape((6,1)) #d-o tersten al ylisti
                    diff = np.matmul(np.diag((np.ones((1,N))-np.square(hlist[j+1].reshape((1,50)))).flatten()),diff) #tanh derivative
                    diff = np.matmul(weight_hh.T,diff)
                    diff = diff + np.matmul(weight_ho.reshape((50,6)),difference)
                    c_updated = c_updated+difference
                    weight_ho_up = weight_ho_up+np.matmul(difference,hlist[j].reshape((1,50)))
                    back_inp = np.matmul(np.diag((np.ones((1,N))-np.square(hlist[j].reshape((1,50)))).flatten()),diff)
                    b_updated =  b_updated+back_inp
                    weight_hh_up = weight_hh_up+np.matmul(back_inp,hlist[j-1].reshape((1,50)))
                    weight_ih_up = weight_ih_up+ np.matmul(back_inp,X[j].reshape((1,3)))

                    #weight_hh_up=10/np.linalg.norm(weight_hh_up)*weight_hh_up
                    #weight_ih_up=10/np.linalg.norm(weight_ih_up)*weight_ih_up
                    #weight_ho_up=10/np.linalg.norm(weight_ho_up)*weight_ho_up
                    #b_updated=10/np.linalg.norm(b_updated)*b_updated
                    #c_updated=10/np.linalg.norm(b_updated)*c_updated
                    

                    #if not np.isnan(weight_hh_up.mean()):
                    weight_ih = weight_ih - learning_rate*weight_ih_up/(10*bs)
                    weight_hh = weight_hh - learning_rate*weight_hh_up/(10*bs)
                    weight_ho = weight_ho - learning_rate*weight_ho_up/(10*bs)

                    b = b - learning_rate*b_updated/(10*bs) 
                    c = c - learning_rate*c_updated/(10*bs)
                    
                    weight_ih = (weight_ih - np.mean(weight_ih, axis=0)) / np.std(weight_ih, axis=0)
                    weight_hh = (weight_hh - np.mean(weight_hh, axis=0)) / np.std(weight_hh, axis=0)
                    weight_ho = (weight_ho - np.mean(weight_ho, axis=0)) / np.std(weight_ho, axis=0)


                    b = (b - np.mean(b, axis=0)) / np.std(b, axis=0)
                    c = (c - np.mean(c, axis=0)) / np.std(c, axis=0)


                mean_error = mean_error+ np.mean(total_error)
        #mean_error= (mean_error - np.mean(mean_error, axis=0)) / np.std(mean_error, axis=0)

    print(mean_error/3000)


Epoch 1
1.4437448149208536
Epoch 2
1.118843177106805
Epoch 3
0.9956398745910906
Epoch 4
0.8411173769699546
Epoch 5
0.9105613481418134
Epoch 6
0.7620315802753324


In [7]:
print(weight_hh)

[[ 0.59181814 -0.59175199  0.59182755 ... -0.59189964  0.59180278
   0.59175827]
 [-2.0361248   2.03606734 -2.03613311 ...  2.0361974  -2.03611121
  -2.03607254]
 [ 0.03816101 -0.03807365  0.03817358 ... -0.03827149  0.03814045
   0.03808166]
 ...
 [-0.00326995  0.00317879 -0.00328301 ...  0.00338172 -0.00324858
  -0.00318715]
 [ 0.63045207 -0.63038988  0.63046091 ... -0.63052828  0.63043762
   0.63039579]
 [ 1.41186554 -1.4118159   1.41187266 ... -1.41192759  1.41185391
   1.41182055]]


In [8]:
np.expand_dims(X[5],axis=0).shape

(1, 3)

In [9]:
equal = 0#find the number of correct predictions d=o
for i in range(3000):
    if (np.argmax(trY[i])==np.argmax(f_propagation(weight_ih,weight_hh,weight_ho,b,c,N,X=trX[i])[0:150].mean(axis=0))):
        equal+=1
accuracy=100*equal/len(trY)
print("Accuracy:", accuracy)
print(equal)

Accuracy: 19.5
585


In [10]:
equal = 0#find the number of correct predictions d=o
for i in range(600):
    if (np.argmax(tstY[i])==np.argmax(f_propagation(weight_ih,weight_hh,weight_ho,b,c,N,X=tstX[i])[0:150].mean(axis=0))):
        equal+=1
accuracy=100*equal/len(tstY)
print("Accuracy:", accuracy)
print(equal)

Accuracy: 14.0
84


In [7]:

for element in range(15): #epoch
    equal = 0 #find the number of correct predictions d=o
    for i in range(3000):
        X=trX_rand[i]
        #f_propagation 
        for t in range(len(X)-1):
            vh = np.dot(weight_ih,np.expand_dims(X[t],axis=0).T) + np.dot(weight_hh,memo)+ b
            hidden = np.tanh(vh)
            vo = np.dot(weight_ho,hidden) + c
            out = sigmoid(vo)
            ylist[t] = out
            memo = hidden
            hlist[t+1] = memo
        if (np.argmax(trY[i])==np.argmax(ylist[0:150].mean(axis=0))):
            equal=equal+ 1
accuracy=100*equal/3000
print("Accuracy:", accuracy)

Accuracy: 16.433333333333334


In [8]:
for element in range(15):
    equal = 0
    for i in range(600):
        X=tstX_rand[i]
        #f_propagation 
        for t in range(len(X)-1):
            vh = np.dot(weight_ih,np.expand_dims(X[t],axis=0).T) + np.dot(weight_hh,memo)+ b
            hidden = np.tanh(vh)
            vo = np.dot(weight_ho,hidden) + c
            out = sigmoid(vo)
            ylist[t] = out
            memo = hidden
            hlist[t+1] = memo
        if (np.argmax(tstY[i])==np.argmax(ylist.mean(axis=0))):
            equal=equal+ 1
accuracy=100*equal/600
print("Accuracy:", accuracy)

Accuracy: 17.166666666666668


In [11]:
def accuracy_top123(weight_ih,weight_hh,weight_ho,b,c,N,tstX,tstY):
    accuracy_top1 = 0
    accuracy_top2 = 0
    accuracy_top3 = 0
    for top in range(600):
        index_array = np.argsort(f_propagation(weight_ih,weight_hh,weight_ho,b,c,N,tstX[top]).mean(axis=0).flatten())
        desired_out = np.argmax(tstY[top])
        if desired_out == index_array[-1]:
            accuracy_top1= accuracy_top1+1
            accuracy_top2= accuracy_top2+1
            accuracy_top3= accuracy_top3+1
            
        

        elif desired_out == index_array[-2]:   
            accuracy_top2= accuracy_top2+1
            accuracy_top3= accuracy_top3+1
            
        elif desired_out == index_array[-3]:  
            accuracy_top3= accuracy_top3+1
    return 100*accuracy_top1/600,100*accuracy_top2/600,100*accuracy_top3/600

In [12]:
accuracy_top123(weight_ih,weight_hh,weight_ho,b,c,N,tstX,tstY)

(14.0, 31.333333333333332, 57.333333333333336)

In [13]:
accuracy_top123(weight_ih,weight_hh,weight_ho,b,c,N,trX,trY)

(3.5, 11.833333333333334, 87.16666666666667)

In [ ]:
############## VALIDATION LOSS FUNCTION

In [18]:
def calculate_validation_loss(X_val, y_val, weight_ih, weight_hh, weight_ho, b, c):
    val_loss = 0
    hlist_val = np.zeros((X_val.shape[0], N, 1))
    ylist_val = np.zeros((X_val.shape[0], 6, 1))

    for i in range(X_val.shape[0]):
        vh1_val = np.matmul(weight_ih, X_val[i].reshape((3, 1)))
        vh_val = vh1_val + np.matmul(weight_hh, hlist_val[i - 1]) + b
        hidden_val = np.tanh(vh_val)
        vo_val = np.matmul(weight_ho, hidden_val) + c
        out_val = sigmoid(vo_val)
        ylist_val[i] = out_val
        hlist_val[i] = hidden_val
        total_error_val = -np.matmul(y_val[i].reshape((1, 6)), np.log(abs(out_val + 1e-8))) - np.matmul(
            (np.ones((1, 6)) - y_val[i]), np.log(abs(np.ones((6, 1)) - out_val) + 1e-8))
        total_error_val = total_error_val.mean()
        if not np.isnan(total_error_val):
            val_loss = val_loss + total_error_val / X_val.shape[0]

    return val_loss

val_loss = calculate_validation_loss(X, tstY[3], weight_ih, weight_hh, weight_ho, b, c)
print("Validation Loss:", val_loss)

ValueError: cannot reshape array of size 1 into shape (1,6)

In [ ]:
################# EARLY STOPPING

In [ ]:
patience = 5
##best_val_loss = 
early_stopping_counter = 0